<a href="https://colab.research.google.com/github/SankethHanasi/BIS_LAB/blob/main/LAB_Cuckoo_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#1BM22CS242 Cuckoo Search Lab Internals
import numpy as np

# Objective Function (to be optimized)
def objective_function(x):
    # Example: simple sum of squares (minimization problem)
    return np.sum(x**2)

# Generate a random Lévy flight step
def levy_flight(D, alpha=1.0):
    # Generate random step based on Lévy flight distribution (simplified)
    u = np.random.normal(0, 1, D)  # Normal distributed random variable u
    v = np.random.normal(0, 1, D)  # Normal distributed random variable v
    step = u / (np.abs(v) ** (1 / alpha))  # Lévy flight step
    return step

# Random initialization of nests in a D-dimensional space
def randomly_initialize(N, D, lower_bound, upper_bound):
    return np.random.uniform(lower_bound, upper_bound, (N, D))

# Evaluate the fitness of each nest
def evaluate_fitness(nests, func):
    return np.apply_along_axis(func, 1, nests)

# Find the index of the nest with the minimum fitness
def min_fitness_index(fitness):
    return np.argmin(fitness)

# Get the worst nests based on fitness
def get_worst_nests(fitness, pa, N):
    sorted_indices = np.argsort(fitness)
    worst_indices = sorted_indices[-int(pa * N):]
    return worst_indices

# Replace the worst nests with random new nests
def replace_worst_nests(nests, worst_nests, N, D, lower_bound, upper_bound):
    for i in worst_nests:
        nests[i] = np.random.uniform(lower_bound, upper_bound, D)
    return nests

# Cuckoo Search Algorithm
def cuckoo_search(Func, D, N, MaxIter, pa, alpha, lower_bound, upper_bound):
    # Initialize nests randomly in D-dimensional space
    nests = randomly_initialize(N, D, lower_bound, upper_bound)

    # Evaluate fitness of each nest
    fitness = evaluate_fitness(nests, Func)

    # Find the best nest so far
    best_nest = nests[min_fitness_index(fitness)]
    best_fitness = np.min(fitness)

    # Main optimization loop
    for iteration in range(MaxIter):
        # Generate new solutions using Lévy flights
        for i in range(N):
            step_size = alpha * levy_flight(D)
            new_nest = nests[i] + step_size
            # Ensure the new nest is within bounds
            new_nest = np.clip(new_nest, lower_bound, upper_bound)
            new_fitness = Func(new_nest)

            # If new nest is better, replace it
            if new_fitness < fitness[i]:
                nests[i] = new_nest
                fitness[i] = new_fitness

        # Abandon worst nests (replace with random new nests)
        worst_nests = get_worst_nests(fitness, pa, N)
        nests = replace_worst_nests(nests, worst_nests, N, D, lower_bound, upper_bound)

        # Update the best nest if needed
        current_best_index = min_fitness_index(fitness)
        if fitness[current_best_index] < best_fitness:
            best_nest = nests[current_best_index]
            best_fitness = fitness[current_best_index]

    return best_nest, best_fitness


# Example usage
if __name__ == "__main__":
    # Problem Parameters
    D = 5  # Dimensionality of the problem
    N = 20  # Number of nests
    MaxIter = 100  # Maximum number of iterations
    pa = 0.25  # Probability of nest abandonment
    alpha = 1.0  # Scaling factor for Lévy flight
    lower_bound = -10  # Lower bound of the search space
    upper_bound = 10  # Upper bound of the search space

    # Running Cuckoo Search
    best_nest, best_fitness = cuckoo_search(objective_function, D, N, MaxIter, pa, alpha, lower_bound, upper_bound)

    print("Best solution:", best_nest)
    print("Best fitness:", best_fitness)


Best solution: [-0.1284373  -0.47110441  0.45869406 -0.41459955 -0.0735937 ]
Best fitness: 0.6261445738348669
